**Part 1**

Scrapping the 20 pages of the prodict listing from 
https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1

Items to scrape
• Product URL
• Product Name
• Product Price
• Rating
• Number of reviews


In [95]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    }

In [8]:
# Dictionary to store the URL of the product pages with the address of the first page.
page_address = {
    "page_1":"https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
    }

In [21]:
# Extracting the URLs of the all remaining 19 pages
for i in range(1, 20):
    # Send an HTTP GET request to the page URL with the appropriate headers
    response = requests.get(page_address[f"page_{i}"], headers=headers)
    
    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find the anchor tag with the aria-label matching the page number
    url = soup.find("a", {"aria-label": f"Go to page {i+1}"})["href"]
    
    # Construct the full URL for the next page
    url = "https://www.amazon.in/" + url
    
    # Store the URL for the next page in the page_address dictionary
    page_address[f"page_{i+1}"] = url

In [49]:
# Initialize empty lists to store product information
p_name = []  # Product names
p_price = []  # Product prices
p_url = []  # Product URLs
p_rating = []  # Product ratings
p_reviews = []  # Number of product reviews

# Iterate through the page addresses in the dictionary
for address in page_address:
    # Send an HTTP GET request to the URL with the provided headers
    response = requests.get(url=page_address[address], headers=headers)
    
    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find all product elements on the page with multiple class names
    for product in soup.find_all("div", class_=["sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16",
                                                "sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 AdHolder sg-col s-widget-spacing-small sg-col-12-of-16"]):
        
        # Find and extract the product name if available, or set it to "N/A"
        product_name_element = product.find("span", class_="a-size-medium a-color-base a-text-normal")
        product_name = product_name_element.text if product_name_element else "N/A"
        
        # Find and construct the product URL if available, or set it to "N/A"
        product_url_element = product.find("a", class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")
        product_url = "https://www.amazon.in" + product_url_element["href"] if product_url_element else "N/A"
        
        # Find and extract the product price if available, or set it to "N/A"
        product_price_element = product.find("span", class_="a-price-whole")
        product_price = product_price_element.text if product_price_element else "N/A"
        
        # Find and extract the number of product reviews if available, or set it to "N/A"
        num_review_element = product.find("span", class_="a-size-base s-underline-text")
        num_review = num_review_element.text if num_review_element else "N/A"
        
        # Find and extract the product rating if available, or set it to "N/A"
        rating_element = product.find("span", class_="a-icon-alt")
        rating = rating_element.text if rating_element else "N/A"
        
        # Append the extracted information to their respective lists
        p_name.append(product_name)
        p_price.append(product_price)
        p_url.append(product_url)
        p_rating.append(rating)
        p_reviews.append(num_review)


In [50]:
# Create a DataFrame to store product information with the provided lists
df = pd.DataFrame({
    'product_name': p_name,     # Product names
    'price (INR)': p_price,    # Product prices
    'rating': p_rating,        # Product ratings
    'URL': p_url,              # Product URLs
    'reviews': p_reviews      # Number of product reviews
})

,product_name,price (INR),rating,URL,reviews
0,URBAN TRIBE Fitpack Neo Multipurpose Laptop Ba...,"1,829",3.8 out of 5 stars,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,660
1,URBAN TRIBE Fitpack Pro Multipurpose Laptop Ba...,"2,289",3.9 out of 5 stars,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,361
2,Wesley Milestone 2.0 Casual Waterproof Laptop ...,499,4.3 out of 5 stars,https://www.amazon.in/Wesley-Milestone-Waterpr...,"13,238"
3,Fur Jaden Brown Textured Leatherette Stylish &...,799,4.3 out of 5 stars,https://www.amazon.in/FUR-JADEN-Leatherette-Po...,"6,886"
4,American Tourister Fizz Large Size 32 Ltrs Cas...,"1,049",4.0 out of 5 stars,https://www.amazon.in/American-Tourister-AMT-S...,"56,357"
...,...,...,...,...,...
421,uppercase 26 Ltrs Ryder Professional Laptop Ba...,"2,400",4.3 out of 5 stars,https://www.amazon.in/UPPERCASE-Professional-B...,97
422,"STRABO Defender Laptop Bag, Light weight & Wat...",999,4.5 out of 5 stars,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,25
423,URBAN TRIBE Havana Laptop Backpack for Men & W...,918,3.9 out of 5 stars,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,821
424,TRAWOC Travel Backpack - 40 Litre Laptop Bag w...,"1,345",5.0 out of 5 stars,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,2


In [52]:
# Save the DataFrame to a CSV file 
df.to_csv('product_details.csv', index=False)